In [1]:
# Run this code when reset session
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import subprocess
!python '/content/gdrive/My Drive/projects/ml_projects/colab_setup.py'

Mounted at /content/gdrive


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/projects/ml_projects')
%load_ext autoreload
%autoreload 2

In [0]:
# Third-party imports
from pandas import read_csv

# Local imports
from ml_projects.apriori_algorithm import AprioriAlgorithm
from ml_projects.helpers import SqlDb

In [0]:
brazilian_ecommerce_df = read_csv(
    'https://github.com/alexjdata/ml_projects/blob/master/ml_projects/sequential_pattern/brazilian_ecommerce_dataset.csv?raw=true'
)

In [0]:
brazilian_ecommerce_db = SqlDb()
brazilian_ecommerce_db.create_table(
    table_name = 'order_line_item',
    data_df = brazilian_ecommerce_df
)

In [0]:
# from pandas import DataFrame as df
dataset_df = brazilian_ecommerce_db.query(
    """
    SELECT order_id, product_category_name_en
    FROM order_line_item
    WHERE
        order_id IN (
            SELECT order_id 
            FROM order_line_item 
            GROUP BY order_id 
            HAVING COUNT(DISTINCT product_category_name_en) > 1
        )
    GROUP BY order_id, product_category_name_en
    ORDER BY order_id
    """
)

In [0]:
dataset = AprioriAlgorithm().set_transaction_dataset(
    transaction_dataset = dataset_df[0:1000],
    transaction_id_column = 'order_id',
    itemset_column = 'product_category_name_en'
).dataset

In [23]:
apriori_algorithm = AprioriAlgorithm(dataset = dataset, minsup = 0.0005, minconf = 0.2)
apriori_algorithm.generate_all_rules()
apriori_algorithm.all_rules

{(frozenset({'music'}), frozenset({'perfumery'})): {'confident': 0.5,
  'lift': 13.7222399,
  'support': 0.0020243},
 (frozenset({'perfumery'}),
  frozenset({'health_beauty'})): {'confident': 0.2777793, 'lift': 3.1187049, 'support': 0.0101215},
 (frozenset({'furniture_living_room'}),
  frozenset({'bed_bath_table'})): {'confident': 0.200001, 'lift': 0.7542022, 'support': 0.0040486},
 (frozenset({'furniture_living_room'}),
  frozenset({'furniture_decor'})): {'confident': 0.400002, 'lift': 1.6065117, 'support': 0.0080972},
 (frozenset({'furniture_living_room'}),
  frozenset({'office_furniture'})): {'confident': 0.200001, 'lift': 14.1143958, 'support': 0.0040486},
 (frozenset({'office_furniture'}),
  frozenset({'furniture_living_room'})): {'confident': 0.2857163, 'lift': 14.1143958, 'support': 0.0040486},
 (frozenset({'christmas_supplies'}), frozenset({'auto'})): {'confident': 1.0,
  'lift': 23.5238214,
  'support': 0.0020243},
 (frozenset({'costruction_tools_garden'}),
  frozenset({'pet_s